# Climate Action Reserve
https://www.climateactionreserve.org

In [1]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### Load data

In [2]:
df = pd.read_csv("../input/Climate_Action_Reserve/homogenised_dataset_CAR.csv")
df.head(1)

,site_id_created,project_id_reported,site_id_reported,site_description_reported,site_sqkm,trees_planted_reported,country,project_description_reported,planting_date_reported,survival_rate_reported,host_name,url,species_count_reported,species_planted_reported,geometry,Creator,project_id_created
0,NaN,CAR1957,NaN,NaN,NaN,NaN,US,The project consists of areas that were signif...,2/8/2023,NaN,Sierra Pacific Industries,NaN,NaN,NaN,NaN,NaN,NaN


### Save it

In [7]:


gdf = gpd.GeoDataFrame(df)  
gdf.to_file('../midsave/climate_action_reserve.gpkg', driver='GPKG')